### 这九个属性分别为：
* 该学生的代码与其他同学代码的最大相似率是否超过相似度阈值（CPMS）
* 该学生的代码与其他同学代码的最大相似率超过或低于阈值的百分比的类别值（CPMSPC）
* 该题是否为考试题（IE）
* 该学生与最大相似率同学所提交的代码通过的测试用例是否相同（CPTC）
* 该学生与最大相似率同学所提交的代码风格相似率（ASSR）
* 算法题的难易程度（DL）
* 该学生的抄题率（PR）
* 该学生抄题的集中度（PCR）
* 该学生刷题成绩的排名（RRB）

In [62]:
import pandas as pd
import numpy as np
import MySQLdb

db=MySQLdb.connect(host='47.95.22.126',port=3306,user='root',passwd='dhuoj@124',db='gdoj',charset='utf8')
c=db.cursor()

In [2]:
# 构造一个专门从数据库当中查询数据的函数
def get_data(sql):
    c.execute(sql)
    rows_list = []
    for row in c.fetchall():
        rows_list.append(row)
    data = pd.DataFrame(data=rows_list)
    return data;

In [3]:
# 查询所有相似度大于0并且相似度小于等于相似度阈值的提交记录，总记录数为39641条
sql = """select s.id, s.similarId as sim_id, s.userid, su.userid as sim_userid, s.similarity, s.sourceCode, su.sourceCode as sim_sourceCode, s.correctCaseIds, su.correctCaseIds as sim_correctCaseIds, p.similarityThreshold, p.difficulty 
from solution s inner join solution su on s.similarId=su.id inner join problems p on s.problemId=p.id and s.similarity<=p.similarityThreshold order by id asc"""
data = get_data(sql)
data.columns = ['sid', 'sim_id', 'uid', 'sim_uid', 'similarity', 'sourceCode', 'sim_sourceCode', 'correctCaseIds', 'sim_correctCaseIds', 'similarityThreshold', 'DL']

In [4]:
print(data.shape)
data.dtypes

(39641, 11)


sid                      int64
sim_id                   int64
uid                      int64
sim_uid                  int64
similarity             float64
sourceCode              object
sim_sourceCode          object
correctCaseIds          object
sim_correctCaseIds      object
similarityThreshold    float64
DL                      object
dtype: object

In [5]:
data.head()

,sid,sim_id,uid,sim_uid,similarity,sourceCode,sim_sourceCode,correctCaseIds,sim_correctCaseIds,similarityThreshold,DL
0,5409,7322,43,190,0.640000,"#include<stdio.h>\nint main(){\n int k,i,a,...","#include<stdio.h>\n\nint main()\n{\n int n,...","1474,1475,1476,1477,1478,1479,1480,","1474,1475,1476,1477,1478,1479,1480,",0.95,易
1,5549,5534,43,71,0.511628,#include<stdio.h>\n#include<math.h>\nint main(...,"#include<stdio.h>\nint main()\n{\n int n,i,...","1481,1482,1483,1484,1485,1486,1487,","1481,1482,1483,1484,1485,1486,1487,",0.85,易
2,5605,5345,31,53,0.857143,#include <stdio.h>\n#define N 40\nint main()\n...,"#include<stdio.h>\nint main()\n{\n int i,a=...","1474,1475,1476,1477,1478,1479,1480,","1474,1475,1476,1477,1478,1479,1480,",0.95,易
3,6134,4991,29,20,0.828571,#include <stdio.h>\n\nint isLeap(int year)\n\n...,#include <stdio.h>\n#include <stdlib.h>\n\nint...,"249,250,251,252,253,254,255,256,257,","249,250,251,252,253,254,255,256,257,",0.90,易
4,6638,5825,50,72,0.752941,#include <stdio.h>\n#include <math.h>\nint zhi...,#include <stdio.h>\n#include <math.h>\nint isP...,"546,547,548,549,550,","546,547,548,549,550,",0.92,易


In [6]:
data.shape[0] - data.count()  # 每列缺失值的个数

sid                    0
sim_id                 0
uid                    0
sim_uid                0
similarity             0
sourceCode             0
sim_sourceCode         0
correctCaseIds         0
sim_correctCaseIds     0
similarityThreshold    0
DL                     0
dtype: int64

In [10]:
data['sid'].value_counts()  # 查看sid是否唯一

149052    1
34106     1
91462     1
30020     1
83266     1
23873     1
109887    1
107838    1
101691    1
100467    1
85283     1
126263    1
130357    1
128308    1
15661     1
73001     1
91430     1
32037     1
70984     1
73033     1
66890     1
68939     1
17762     1
89441     1
87392     1
48477     1
82435     1
105817    1
38232     1
60759     1
         ..
119417    1
21247     1
23294     1
94971     1
31482     1
25337     1
139172    1
137974    1
114280    1
13043     1
84397     1
117516    1
62251     1
115469    1
58153     1
60200     1
37671     1
105254    1
35620     1
135669    1
107297    1
21279     1
19228     1
31514     1
123141    1
131861    1
15122     1
103552    1
121614    1
133121    1
Name: sid, Length: 39641, dtype: int64

In [7]:
# print(data.shape)
# data_label = data.dropna(subset=['similarId'])  # 将similarId为空的记录删除
# data_label['similarId'] = data_label['similarId'].astype('int')
data_label=data
print(data_label.shape)

(39641, 11)


In [8]:
# 在上面的基础上，筛选出具有标签值的记录
# data_label = get_data("""select s.id, s.userid, s.similarId, s.similarity, p.similarityThreshold, p.difficulty, sw.submited from solution s INNER JOIN 
# problems p on s.similarity>0 and s.problemId=p.id INNER JOIN similaritywarning sw on sw.submited>=0 and s.id=sw.solutionId""")

# data_label.columns=['sid', 'uid', 'similarId', 'similarity', 'similarityThreshold', 'DL', 'LABEL']

In [9]:
# 不抄袭的肯定比抄袭的要多很多，总记录数为3922条
# print(data_label.shape)
# print(len(data_label[data_label['LABEL']==1]))  # label=1代表抄袭
# print(len(data_label[data_label['LABEL']==0]))  # label=0代表不抄袭
# data_label.head()

In [63]:
# 查询用户排名数据
data_user_rank = get_data("""select userid, SUM(score) as allscore from solution where similarity>0 GROUP BY userid ORDER BY allscore desc""")
data_user_rank.columns = ['uid', 'total_score']

In [64]:
print(data_user_rank.shape)
data_user_rank.head()

(481, 2)


,uid,total_score
0,132,1390.000000
1,88,963.100002
2,126,949.460001
3,1218,914.500000
4,130,823.960001


In [14]:
# 在label列前面插入rank列
dl_col_name = data_label.columns.tolist()
dl_col_name.insert(dl_col_name.index('DL'), 'CPMS')
dl_col_name.insert(dl_col_name.index('DL'), 'CPMSPC')
# dl_col_name.insert(dl_col_name.index('DL'), 'IE')  # 对于solution表中的数据每一个problem都有一场exam与之对应
dl_col_name.insert(dl_col_name.index('DL'), 'CPTC')
dl_col_name.insert(dl_col_name.index('DL'), 'ASSR')
dl_col_name.insert(dl_col_name.index('DL')+1, 'PR')
dl_col_name.insert(dl_col_name.index('PR')+1, 'PCR')
dl_col_name.insert(dl_col_name.index('PCR')+1, 'RRB')
data_label = data_label.reindex(columns=dl_col_name)
# data_label['CPMS'] = data_label['CPMS'].astype('int32')
# data_label['CPMSPC'] = data_label['CPMSPC'].astype('int32')
# data_label['CPTC'] = data_label['CPTC'].astype('int32')
# data_label['DL'] = data_label['DL'].astype('int32')
data_label.head()

,sid,sim_id,uid,sim_uid,similarity,sourceCode,sim_sourceCode,correctCaseIds,sim_correctCaseIds,similarityThreshold,CPMS,CPMSPC,CPTC,ASSR,DL,PR,PCR,RRB
0,5409,7322,43,190,0.640000,"#include<stdio.h>\nint main(){\n int k,i,a,...","#include<stdio.h>\n\nint main()\n{\n int n,...","1474,1475,1476,1477,1478,1479,1480,","1474,1475,1476,1477,1478,1479,1480,",0.95,NaN,NaN,NaN,NaN,易,NaN,NaN,NaN
1,5549,5534,43,71,0.511628,#include<stdio.h>\n#include<math.h>\nint main(...,"#include<stdio.h>\nint main()\n{\n int n,i,...","1481,1482,1483,1484,1485,1486,1487,","1481,1482,1483,1484,1485,1486,1487,",0.85,NaN,NaN,NaN,NaN,易,NaN,NaN,NaN
2,5605,5345,31,53,0.857143,#include <stdio.h>\n#define N 40\nint main()\n...,"#include<stdio.h>\nint main()\n{\n int i,a=...","1474,1475,1476,1477,1478,1479,1480,","1474,1475,1476,1477,1478,1479,1480,",0.95,NaN,NaN,NaN,NaN,易,NaN,NaN,NaN
3,6134,4991,29,20,0.828571,#include <stdio.h>\n\nint isLeap(int year)\n\n...,#include <stdio.h>\n#include <stdlib.h>\n\nint...,"249,250,251,252,253,254,255,256,257,","249,250,251,252,253,254,255,256,257,",0.90,NaN,NaN,NaN,NaN,易,NaN,NaN,NaN
4,6638,5825,50,72,0.752941,#include <stdio.h>\n#include <math.h>\nint zhi...,#include <stdio.h>\n#include <math.h>\nint isP...,"546,547,548,549,550,","546,547,548,549,550,",0.92,NaN,NaN,NaN,NaN,易,NaN,NaN,NaN


In [15]:
# 计算rank的值
def get_index_from_data_user_rank(data):
    i = data_user_rank[data_user_rank['uid']==data['uid']].index.tolist()
    return i[0]

# 计算CPMS的值
def get_cpms(data):
    if data['similarity']>data['similarityThreshold']:
        return 1
    else:
        return 0
    
# 将DL值转化为数值类型
def change_dl(data):
    if data['DL']=='易':
        return 1
    elif data['DL']=='中':
        return 2
    else:
        return 3
    
# 将CPMSPC值进行数值化
def change_cpmspc(data):
    rate=np.abs(data['similarity']-data['similarityThreshold'])/data['similarityThreshold']
    if data['CPMS']==0:  # 表示最大相似率低于相似度阈值
        if rate>0.6 and rate<=1:
            return 1
        elif rate>0.3 and rate<=0.6:
            return 2
        elif rate>0.1 and rate<=0.3:
            return 3
        else:
            return 4
    elif data['CPMS']==1:  # 表示最大相似率高于相似度阈值
        if rate>0.6 and rate<=1:
            return 8
        elif rate>0.3 and rate<=0.6:
            return 7
        elif rate>0.1 and rate<=0.3:
            return 6
        else:
            return 5
        
# 将CPTC(测试用例是否相同)的值进行数值化：0/1
def change_cptc(data):  # 该data其实表示的是原始数据当中的一行数据
    if data['correctCaseIds'] == data['sim_correctCaseIds']:
        return 1
    return 0

# 将一份代码当中的括弧组合成一个字符串
# 将一份代码当中的空格、缩进、空行组合成一个字符串
# 返回这两个字符串
def count_brackets_and_char(code):
    result_brackets="";
    result_char="";
    s_brackets = set('(){}[]')
    s_char = set("""'\x20''\x09''\x0a'""")
    for ch in code:
        if ch in s_brackets:
            result_brackets=result_brackets+ch
        elif ch in s_char:
            result_char=result_char+ch
    return result_brackets, result_char

# 借用lcs算法设计思想，计算括弧的相似率
def cal_bsr(a, b):
    if (a==None or a=='') or (b==None or b==''):
        return 0.0
    lena=len(a)
    lenb=len(b)
    c=[[0 for i in range(lenb+1)] for j in range(lena+1)]
    for i in range(lena):
        for j in range(lenb):
            if a[i]==b[j]:
                c[i+1][j+1]=c[i][j]+1
            elif c[i+1][j]>c[i][j+1]:
                c[i+1][j+1]=c[i+1][j]
            else:
                c[i+1][j+1]=c[i][j+1]
    return (2*c[lena][lenb])/(lena+lenb)

# 使用lcs计算WPSR相似度
def cal_wpsr(a, b):
    if (a==None or a=='') or (b==None or b==''):
        return 0.0
    lena=len(a)
    lenb=len(b)
    c=[[0 for i in range(lenb+1)] for j in range(lena+1)]
    for i in range(lena):
        for j in range(lenb):
            if a[i]==b[j]:
                c[i+1][j+1]=c[i][j]+1
            elif c[i+1][j]>c[i][j+1]:
                c[i+1][j+1]=c[i+1][j]
            else:
                c[i+1][j+1]=c[i][j+1]
    return (2*c[lena][lenb])/(lena+lenb)

def cal_assr(data):
    a=data['sourceCode']
    b=data['sim_sourceCode']
    a_brackets, a_chars=count_brackets_and_char(a)
    b_brackets, b_chars=count_brackets_and_char(b)
    WPSR=cal_wpsr(a_chars, b_chars)
    BSR=cal_bsr(a_brackets, b_brackets)
    return (WPSR+BSR)/2

In [19]:
data_label['RRB'] = data_label.apply(func=get_index_from_data_user_rank, axis=1)
data_label['CPMS'] = data_label.apply(func=get_cpms, axis=1)
data_label['DL'] = data_label.apply(func=change_dl, axis=1)
data_label['CPMSPC'] = data_label.apply(func=change_cpmspc, axis=1)

In [20]:
data_label['CPTC'] = data_label.apply(func=change_cptc, axis=1)

In [23]:
data_label['ASSR'] = data_label.apply(func=cal_assr, axis=1)

In [24]:
print(data_label.shape)
data_label.head()

(39641, 18)


,sid,sim_id,uid,sim_uid,similarity,sourceCode,sim_sourceCode,correctCaseIds,sim_correctCaseIds,similarityThreshold,CPMS,CPMSPC,CPTC,ASSR,DL,PR,PCR,RRB
0,5409,7322,43,190,0.640000,"#include<stdio.h>\nint main(){\n int k,i,a,...","#include<stdio.h>\n\nint main()\n{\n int n,...","1474,1475,1476,1477,1478,1479,1480,","1474,1475,1476,1477,1478,1479,1480,",0.95,0,2,1,0.882609,1,NaN,NaN,228
1,5549,5534,43,71,0.511628,#include<stdio.h>\n#include<math.h>\nint main(...,"#include<stdio.h>\nint main()\n{\n int n,i,...","1481,1482,1483,1484,1485,1486,1487,","1481,1482,1483,1484,1485,1486,1487,",0.85,0,2,1,0.627778,1,NaN,NaN,228
2,5605,5345,31,53,0.857143,#include <stdio.h>\n#define N 40\nint main()\n...,"#include<stdio.h>\nint main()\n{\n int i,a=...","1474,1475,1476,1477,1478,1479,1480,","1474,1475,1476,1477,1478,1479,1480,",0.95,0,4,1,0.967105,1,NaN,NaN,337
3,6134,4991,29,20,0.828571,#include <stdio.h>\n\nint isLeap(int year)\n\n...,#include <stdio.h>\n#include <stdlib.h>\n\nint...,"249,250,251,252,253,254,255,256,257,","249,250,251,252,253,254,255,256,257,",0.90,0,4,1,0.775883,1,NaN,NaN,174
4,6638,5825,50,72,0.752941,#include <stdio.h>\n#include <math.h>\nint zhi...,#include <stdio.h>\n#include <math.h>\nint isP...,"546,547,548,549,550,","546,547,548,549,550,",0.92,0,3,1,0.932974,1,NaN,NaN,123


In [26]:
data_label_copy=data_label.copy()
data_label_copy.head()

,sid,sim_id,uid,sim_uid,similarity,sourceCode,sim_sourceCode,correctCaseIds,sim_correctCaseIds,similarityThreshold,CPMS,CPMSPC,CPTC,ASSR,DL,PR,PCR,RRB
0,5409,7322,43,190,0.640000,"#include<stdio.h>\nint main(){\n int k,i,a,...","#include<stdio.h>\n\nint main()\n{\n int n,...","1474,1475,1476,1477,1478,1479,1480,","1474,1475,1476,1477,1478,1479,1480,",0.95,0,2,1,0.882609,1,NaN,NaN,228
1,5549,5534,43,71,0.511628,#include<stdio.h>\n#include<math.h>\nint main(...,"#include<stdio.h>\nint main()\n{\n int n,i,...","1481,1482,1483,1484,1485,1486,1487,","1481,1482,1483,1484,1485,1486,1487,",0.85,0,2,1,0.627778,1,NaN,NaN,228
2,5605,5345,31,53,0.857143,#include <stdio.h>\n#define N 40\nint main()\n...,"#include<stdio.h>\nint main()\n{\n int i,a=...","1474,1475,1476,1477,1478,1479,1480,","1474,1475,1476,1477,1478,1479,1480,",0.95,0,4,1,0.967105,1,NaN,NaN,337
3,6134,4991,29,20,0.828571,#include <stdio.h>\n\nint isLeap(int year)\n\n...,#include <stdio.h>\n#include <stdlib.h>\n\nint...,"249,250,251,252,253,254,255,256,257,","249,250,251,252,253,254,255,256,257,",0.90,0,4,1,0.775883,1,NaN,NaN,174
4,6638,5825,50,72,0.752941,#include <stdio.h>\n#include <math.h>\nint zhi...,#include <stdio.h>\n#include <math.h>\nint isP...,"546,547,548,549,550,","546,547,548,549,550,",0.92,0,3,1,0.932974,1,NaN,NaN,123


In [27]:
data_label_delete = data_label_copy.drop(['sourceCode', 'sim_sourceCode', 'correctCaseIds', 'sim_correctCaseIds'], axis=1)
data_label_delete.head()

,sid,sim_id,uid,sim_uid,similarity,similarityThreshold,CPMS,CPMSPC,CPTC,ASSR,DL,PR,PCR,RRB
0,5409,7322,43,190,0.640000,0.95,0,2,1,0.882609,1,NaN,NaN,228
1,5549,5534,43,71,0.511628,0.85,0,2,1,0.627778,1,NaN,NaN,228
2,5605,5345,31,53,0.857143,0.95,0,4,1,0.967105,1,NaN,NaN,337
3,6134,4991,29,20,0.828571,0.90,0,4,1,0.775883,1,NaN,NaN,174
4,6638,5825,50,72,0.752941,0.92,0,3,1,0.932974,1,NaN,NaN,123


In [28]:
data_label_delete.shape

(39641, 14)

In [53]:
data_label_delete=data_label_delete[data_label_delete['similarity']!=-1.0]

In [65]:
data_label_delete['RRB'] = data_label_delete.apply(func=get_index_from_data_user_rank, axis=1)

In [66]:
data_label_delete.shape

(22842, 14)

In [77]:
data_user_rank.iloc[407]

uid            1241.00
total_score      27.32
Name: 407, dtype: float64

In [152]:
data_label_delete.head(10)

,sid,sim_id,uid,sim_uid,similarity,similarityThreshold,CPMS,CPMSPC,CPTC,ASSR,DL,PR,PCR,RRB
0,5409,7322,43,190,0.640000,0.95,0,2,1,0.882609,1,NaN,0.631714,163
1,5549,5534,43,71,0.511628,0.85,0,2,1,0.627778,1,NaN,0.631714,163
2,5605,5345,31,53,0.857143,0.95,0,4,1,0.967105,1,NaN,0.565435,145
3,6134,4991,29,20,0.828571,0.90,0,4,1,0.775883,1,NaN,0.813397,188
4,6638,5825,50,72,0.752941,0.92,0,3,1,0.932974,1,NaN,0.703248,169
5,6915,6597,28,24,0.784314,0.92,0,3,1,0.888374,1,NaN,0.586463,128
6,7122,5931,67,69,0.705882,0.96,0,3,1,0.640446,1,NaN,0.659596,108
7,7201,6965,61,81,0.646154,0.92,0,3,1,0.814841,2,NaN,0.716036,151
8,7314,5469,191,81,0.785714,0.95,0,3,1,0.943709,1,NaN,0.565217,85
9,7348,7322,182,190,0.857143,0.95,0,4,1,0.944444,1,NaN,0.123810,222


In [160]:
pcr_main_dict[182]

0.12380952380952381

In [154]:
data_label_delete.to_csv('D:/fgz/专利/OJFeatures.csv', index=False)

In [155]:
data_label_delete.dtypes

sid                      int64
sim_id                   int64
uid                      int64
sim_uid                  int64
similarity             float64
similarityThreshold    float64
CPMS                     int64
CPMSPC                   int64
CPTC                     int64
ASSR                   float64
DL                       int64
PR                     float64
PCR                    float64
RRB                      int64
dtype: object

In [81]:
data_label_delete['sid'].count()

22842

In [153]:
data_label_delete.shape[0] - data_label_delete.count()

sid                        0
sim_id                     0
uid                        0
sim_uid                    0
similarity                 0
similarityThreshold        0
CPMS                       0
CPMSPC                     0
CPTC                       0
ASSR                       0
DL                         0
PR                     22842
PCR                        0
RRB                        0
dtype: int64

In [83]:
pcr_set=data_label_delete['sim_uid'].groupby(data_label_delete['uid'])

In [89]:
print(type(pcr_set.value_counts()))
print(pcr_set.value_counts())

<class 'pandas.core.series.Series'>
uid   sim_uid
1     45         9
      1024       7
      1042       4
      38         2
      55         2
      66         2
      130        2
      161        2
      1003       2
      1033       2
      1100       2
      23         1
      30         1
      37         1
      59         1
      75         1
      77         1
      106        1
      128        1
      150        1
      154        1
      1016       1
      1048       1
      1054       1
      1058       1
      1089       1
17    51         6
      57         4
      75         4
      52         3
                ..
1279  210        1
      1016       1
      1018       1
      1028       1
      1278       1
1281  99         1
      127        1
      1192       1
1284  146        2
      57         1
      1008       1
      1023       1
      1052       1
      1103       1
      1270       1
1286  37         1
      50         1
      55         1
      133        1


In [119]:
pcr_dict=pcr_set.value_counts(ascending=True).to_dict()

In [120]:
pcr_dict

{(1, 23): 1,
 (1, 30): 1,
 (1, 37): 1,
 (1, 59): 1,
 (1, 75): 1,
 (1, 77): 1,
 (1, 106): 1,
 (1, 128): 1,
 (1, 150): 1,
 (1, 154): 1,
 (1, 1016): 1,
 (1, 1048): 1,
 (1, 1054): 1,
 (1, 1058): 1,
 (1, 1089): 1,
 (1, 38): 2,
 (1, 55): 2,
 (1, 66): 2,
 (1, 130): 2,
 (1, 161): 2,
 (1, 1003): 2,
 (1, 1033): 2,
 (1, 1100): 2,
 (1, 1042): 4,
 (1, 1024): 7,
 (1, 45): 9,
 (17, 24): 1,
 (17, 28): 1,
 (17, 32): 1,
 (17, 35): 1,
 (17, 42): 1,
 (17, 44): 1,
 (17, 47): 1,
 (17, 55): 1,
 (17, 65): 1,
 (17, 70): 1,
 (17, 87): 1,
 (17, 126): 1,
 (17, 130): 1,
 (17, 143): 1,
 (17, 144): 1,
 (17, 150): 1,
 (17, 155): 1,
 (17, 184): 1,
 (17, 186): 1,
 (17, 27): 2,
 (17, 59): 2,
 (17, 73): 2,
 (17, 76): 2,
 (17, 81): 2,
 (17, 52): 3,
 (17, 61): 3,
 (17, 78): 3,
 (17, 57): 4,
 (17, 75): 4,
 (17, 51): 6,
 (18, 26): 1,
 (18, 1108): 1,
 (19, 1035): 1,
 (19, 1057): 1,
 (19, 1214): 1,
 (20, 1): 1,
 (20, 17): 1,
 (20, 27): 1,
 (20, 31): 1,
 (20, 32): 1,
 (20, 35): 1,
 (20, 40): 1,
 (20, 48): 1,
 (20, 49): 1,
 (20,

In [121]:
pcr_series=pcr_set.value_counts(ascending=True)

In [146]:
pcr_series.iloc[0]

1

In [131]:
nunique_series=pcr_set.nunique()

In [134]:
nunique_series

uid
1       26
17      30
18       2
19       3
20      41
21      49
22      32
23      59
24      37
25      32
26      23
27      50
28      41
29      28
30      36
31      47
32      44
33      40
34      38
35      56
36      33
37      41
38      44
39      25
40      43
41      42
42      40
43      37
44      36
45      40
        ..
1250    21
1251    11
1252    23
1253     2
1258     1
1259     4
1260    11
1263     7
1264    14
1265     4
1266     1
1267     1
1268    12
1269     1
1270    13
1271     7
1272     1
1273     2
1274     1
1275    10
1276     2
1277     9
1278    13
1279    12
1281     3
1284     7
1286     6
1288     1
1290     3
1291     5
Name: sim_uid, Length: 481, dtype: int64

In [145]:
nunique_series.iloc[0]

26

In [104]:
pcr_main_dict=pcr_set.size().to_dict()

In [108]:
pcr_main_dict

{1: 51,
 17: 52,
 18: 2,
 19: 3,
 20: 51,
 21: 71,
 22: 52,
 23: 102,
 24: 58,
 25: 60,
 26: 28,
 27: 98,
 28: 71,
 29: 76,
 30: 68,
 31: 77,
 32: 73,
 33: 64,
 34: 64,
 35: 112,
 36: 79,
 37: 65,
 38: 88,
 39: 64,
 40: 63,
 41: 77,
 42: 79,
 43: 68,
 44: 63,
 45: 84,
 46: 59,
 47: 56,
 48: 68,
 49: 71,
 50: 67,
 51: 55,
 52: 98,
 53: 104,
 55: 98,
 56: 57,
 57: 112,
 58: 80,
 59: 80,
 60: 73,
 61: 74,
 62: 94,
 63: 71,
 64: 79,
 65: 48,
 66: 53,
 67: 99,
 68: 37,
 69: 70,
 70: 56,
 71: 73,
 72: 74,
 73: 81,
 74: 82,
 75: 61,
 76: 71,
 77: 76,
 78: 100,
 79: 73,
 80: 49,
 81: 73,
 82: 55,
 84: 4,
 85: 1,
 86: 12,
 87: 39,
 88: 75,
 89: 32,
 90: 71,
 91: 1,
 92: 33,
 93: 51,
 94: 18,
 95: 6,
 96: 5,
 97: 27,
 98: 44,
 99: 25,
 100: 29,
 101: 56,
 102: 59,
 103: 35,
 104: 30,
 105: 34,
 106: 61,
 107: 20,
 108: 31,
 109: 35,
 110: 11,
 111: 31,
 112: 34,
 115: 43,
 116: 32,
 117: 31,
 118: 71,
 119: 45,
 120: 62,
 121: 63,
 122: 39,
 123: 56,
 124: 19,
 125: 40,
 126: 66,
 127: 33,
 128:

In [147]:
pcr_main_dict_copy=pcr_main_dict.copy()
j=0
k=0
for i in pcr_main_dict_copy:
    temp=pcr_main_dict_copy[i]
    N=(1+temp)*temp/2
    fm=0
    for m in range(nunique_series.iloc[j]):
        fm=fm+pcr_series.iloc[k]*(nunique_series.iloc[j]-m)
        k=k+1
    pcr_main_dict[i]=1-fm/N
    j=j+1

In [149]:
print(len(pcr_main_dict))
pcr_main_dict

481


{1: 0.6681749622926094,
 17: 0.5936139332365747,
 18: 0.0,
 19: 0.0,
 20: 0.31598793363499245,
 21: 0.47887323943661975,
 22: 0.5740203193033382,
 23: 0.5912811726632401,
 24: 0.5113968439509059,
 25: 0.6453551912568306,
 26: 0.28325123152709364,
 27: 0.6796536796536796,
 28: 0.5864632237871674,
 29: 0.8133971291866029,
 30: 0.618073316283035,
 31: 0.5654345654345654,
 32: 0.5827471306923362,
 33: 0.5682692307692307,
 34: 0.573076923076923,
 35: 0.688527180783818,
 36: 0.7829113924050632,
 37: 0.5328671328671328,
 38: 0.6710929519918284,
 39: 0.8052884615384616,
 40: 0.4776785714285714,
 41: 0.6213786213786214,
 42: 0.6636075949367088,
 43: 0.6317135549872123,
 44: 0.5952380952380952,
 45: 0.7179271708683473,
 46: 0.6864406779661016,
 47: 0.5726817042606516,
 48: 0.7088661551577153,
 49: 0.6451486697965572,
 50: 0.7032484635645303,
 51: 0.6318181818181818,
 52: 0.6775922490208204,
 53: 0.7551282051282051,
 55: 0.7037724180581324,
 56: 0.557773744706594,
 57: 0.7368836915297092,
 58: 0.

In [150]:
def cal_pcr(data):
    return pcr_main_dict[data['uid']]

In [151]:
data_label_delete['PCR'] = data_label_delete.apply(func=cal_pcr, axis=1)

In [86]:
data_label_delete[data_label_delete['uid']==1291]

,sid,sim_id,uid,sim_uid,similarity,similarityThreshold,CPMS,CPMSPC,CPTC,ASSR,DL,PR,PCR,RRB
21616,83430,80642,1291,1048,0.614420,0.95,0,2,1,0.774111,2,NaN,NaN,397
21627,83478,33984,1291,61,0.578431,0.92,0,2,1,0.717327,1,NaN,NaN,397
21630,83489,30160,1291,119,0.537313,0.95,0,2,1,0.666817,1,NaN,NaN,397
21634,83503,80257,1291,1086,0.464286,0.95,0,2,1,0.719162,1,NaN,NaN,397
21636,83508,31199,1291,126,0.606061,0.96,0,2,1,0.821162,1,NaN,NaN,397


In [18]:
data_label.dtypes

sid                      int64
uid                      int64
similarId                int64
similarity             float64
sourceCode              object
sim_sourceCode          object
correctCaseIds          object
sim_correctCaseIds      object
similarityThreshold    float64
CPMS                     int64
CPMSPC                   int64
CPTC                     int64
ASSR                   float64
DL                       int64
PR                     float64
PCR                    float64
RRB                      int64
dtype: object

In [25]:
# data_label['DL'].value_counts()  # 统计一列当中可能出现的值的个数
data_label['similarId'].count()

21936

In [161]:
data_label.shape[0] - data_label.count()  # 每列缺失值的个数

sid                        0
sim_id                     0
uid                        0
sim_uid                    0
similarity                 0
sourceCode                 0
sim_sourceCode             0
correctCaseIds             0
sim_correctCaseIds         0
similarityThreshold        0
CPMS                       0
CPMSPC                     0
CPTC                       0
ASSR                       0
DL                         0
PR                     39641
PCR                    39641
RRB                        0
dtype: int64

In [21]:
data_label['CPTC'].value_counts()

1    20825
0     1111
Name: CPTC, dtype: int64

In [28]:
# data_label.head()
print(type(data_label))

<class 'NoneType'>


In [41]:
# 选中整数型的列
df_int_columns = data_label.select_dtypes(include=['int64'])

In [42]:
# 向下转型
df_int_columns = df_int_columns.apply(pd.to_numeric, downcast='unsigned')

In [43]:
df_int_columns.dtypes

sid    uint32
uid    uint32
dtype: object

In [44]:
data_label.loc[:, 'uid']=df_int_columns['uid']

C:\Anaconda\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [3]:
def count_brackets_and_char(code):
    result_brackets="";
    result_char="";
    s_brackets = set('(){}[]')
    s_char = set("""'\x20''\x09''\x0a'""")
    for ch in code:
        if ch in s_brackets:
            result_brackets=result_brackets+ch
        elif ch in s_char:
            result_char=result_char+ch
    return result_brackets, result_char

str1="""#include <stdio.h>

int main()
{
    int n;
    float a[40];
    while(1==scanf("%d",&n))
    {
        if(n==0)
        {
            printf("0\n");
            continue;
        }
        int i,count=0;
        for(i=0;i<n;i++)
        {
            scanf("%f",a+i);
        }
        for(i=0;i<n;i++)
        {
            if(a[i]<60) count=count+1;
        }
        printf("%d\n",count);
    }
}
"""
str2="""#include <stdio.h>
#include <stdlib.h>
int main()
{
    int n;
    float A[40];
    while(scanf("%d",&n)==1)
    {
        if(n==0)
        {
            printf("0\n");
            continue;
        }
        int i,m=0;
        for(i=0;i<n;i++)
        {
            scanf("%f",A+i);
        }
        for(i=0;i<n;i++)
        {
            if(A[i]<60) m=m+1;
        }
        printf("%d\n",m);
    }
}
"""

brackets, chars = count_brackets_and_char(str1)
print(brackets)
print(chars)

(){[](()){(){()}(){()}(){([])}()}}
 

 

     
     
    
    
        
        
            

            
        
         
        
        
            
        
        
        
             
        
        

    


